<a href="https://colab.research.google.com/github/jscienciadados/visualizacao-dados-plotly/blob/main/03_Graficos_Espaciais_Mapas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# <font color='blue'>Visualização de Dados e Design de Dashboards</font>
## <font color='blue'>Capítulo 2 - Métodos de Visualização</font>

## Exemplos de Gráficos Espaciais e Mapas

## Pacotes Python Para Manipulação de Dados e Visualização

In [4]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [5]:
# Manipulação de Dados
import numpy as np
import pandas as pd

# Plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots

# Supress warnings
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [6]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Graficos Espaciais" --iversions

Author: Graficos Espaciais

pandas : 1.1.5
IPython: 5.5.0
plotly : 4.4.1
numpy  : 1.19.5
sys    : 3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0]



## Carregando os Dados

In [7]:
# Carrega os dados
shoot = pd.read_csv('dados/police-violence-in-the-us-shootings_wash_post.csv')
shoot.head(3)

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False


In [8]:
# Carrega os dados
terror = pd.read_csv('dados/globalterrorismdb_0718dist.csv', encoding = 'ISO-8859-1')
terror.head(3)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types.Specify dtype option on import or set low_memory=False.



,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,...,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,Central America & Caribbean,NaN,Santo Domingo,18.456792,-69.951164,1.0,0,NaN,NaN,1,1,1,0.0,NaN,NaN,0.0,1,0,1,Assassination,NaN,NaN,NaN,NaN,14,Private Citizens & Property,68.0,Named Civilian,NaN,Julio Guzman,...,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,North America,Federal,Mexico city,19.371887,-99.086624,1.0,0,NaN,NaN,1,1,1,0.0,NaN,NaN,0.0,1,0,6,Hostage Taking (Kidnapping),NaN,NaN,NaN,NaN,7,Government (Diplomatic),45.0,"Diplomatic Personnel (outside of embassy, cons...",Belgian Ambassador Daughter,"Nadine Chaval, daughter",...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,NaN,Mexico,1.0,800000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,4.0,0,NaN,NaN,1,1,1,0.0,NaN,NaN,0.0,1,0,1,Assassination,NaN,NaN,NaN,NaN,10,Journalists & Media,54.0,Radio Journalist/Staff/Facility,Voice of America,Employee,...,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [9]:
# Carrega os dados
brazil = pd.read_csv('dados/brazilian-ecommerce-olist_geolocation_dataset.csv')
brazil.head(3)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP


## Choropleth Maps

**Objetivo**: Exibir mapa de calor por contagem.

Qual o volume de mortes de pessoas negras em todos os estados dos EUA?

In [10]:
# Prepara os dados
black_state = shoot[shoot['race']=='B']['state'].value_counts().to_frame().reset_index().rename(columns = {'index':'state','state':'count'})

In [11]:
# Figura
fig = go.Figure(go.Choropleth(locations = black_state['state'],
                              z = black_state['count'].astype(float),
                              locationmode = 'USA-states',
                              colorscale = 'Reds',
                              autocolorscale = False,
                              text = black_state['state'], 
                              marker_line_color = 'white', 
                              colorbar_title = "Milhões USD",
                              showscale = False,))

# Layout
fig.update_layout(title_text = 'Mortes de Negros Por Estado (2015-2020)',
                  title_x = 0.5,
                  geo = dict(scope = 'usa',
                             projection = go.layout.geo.Projection(type = 'albers usa'),
                             showlakes = True, 
                             lakecolor = 'rgb(255, 255, 255)'))

# Template de cores
fig.update_layout(template = "plotly_dark")

# Gráfico
fig.show()

### Mapbox com Linhas

**Objetivo**: Exibir limites e densidade por linhas usando latitude e longitude.

Como as fronteiras do comércio eletrônico são estendidas para três estados do Brasil - AC, MA e AP?

In [12]:
# Prepara os dados
states = brazil[(brazil['geolocation_state']=='AC')|(brazil['geolocation_state']=='MA')|(brazil['geolocation_state']=='AP')]

In [13]:
# Figura
fig = px.line_mapbox(states, 
                     lat = "geolocation_lat", 
                     lon = "geolocation_lng", 
                     color = "geolocation_state", 
                     zoom = 3, 
                     height = 300)

# Layout
fig.update_layout(mapbox_style = "stamen-terrain", 
                  mapbox_zoom = 2,
                  mapbox_center_lat = -11,
                  margin = {"r":0,"t":0,"l":0,"b":0})

# Gráfico
fig.show()

### Bubble Maps

**Objetivo**: Exibir densidade de valores com bolhas sobre latitude e longitude.

Quantas pessoas foram mortas em ataques terroristas em cada estado dos EUA?

In [14]:
# Prepara os dados
us_terror = terror.loc[(terror['country_txt']=='United States') & (terror['provstate']!='Unknown')][['provstate','latitude','longitude','nkill']]
map_terror = us_terror.groupby(['provstate']).agg({'nkill':'sum', 'latitude':'mean','longitude':'mean'}).reset_index()

In [15]:
# Parâmetros para os gráficos
limits = [(0,5), (6,15), (16,3000)]
colors = ["royalblue", "crimson", "lightseagreen"]
cities = []
scale = 1

In [16]:
# Figura
fig = go.Figure()

# Loop para configuração dos parâmetros associados aos dados
for i in range(len(limits)):
    lim = limits[i]
    df_sub = map_terror[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(locationmode = 'USA-states',
                                lon = df_sub['longitude'],
                                lat = df_sub['latitude'],
                                marker = dict(size = df_sub['nkill'] / scale,
                                              color = colors[i],
                                              line_color = 'rgb(40,40,40)',
                                              line_width = 0.5,
                                              sizemode = 'area'),
        name = '{0} - {1}'.format(lim[0],lim[1])))

fig.update_layout(title_text = 'Ataques Terroristas nos EUA',
                  title_x = 0.5,
                  showlegend = True,
                  geo = dict(scope = 'usa',
                             landcolor = 'rgb(217, 217, 217)'))

# Gráfico
fig.show()

### Mapbox Density

**Objetivo**: Exibir densidade de valores com mapa de calor sobre latitude e longitude.

Quantas pessoas foram mortas em ataques terroristas em todo o mundo?

In [17]:
# Prepara os dados
all_terror = terror[['city','latitude','longitude','nkill']]
map_all_terror  =all_terror.groupby(['latitude','longitude']).agg({'nkill':'sum'}).reset_index()

In [18]:
# Mapbox
fig = px.density_mapbox(map_all_terror, 
                        lat = 'latitude', 
                        lon = 'longitude', 
                        z = 'nkill', 
                        radius = 10,
                        center = dict(lat = 31, lon = 36), zoom = 1, mapbox_style = "stamen-terrain")

# Layout
fig.update_layout(title_text = 'Pessoas Mortas em Ataques Terroristas ao Redor do Mundo',
                  title_x = 0.5, 
                  showlegend = True)

# Gráfico
fig.show()

### Mapbox Layers

**Objetivo**: Exibir pontos de latitude e longitude.

Qual o total de lojas nos estados RO, AM, AC, AP, RR no Brasil?

In [19]:
# Prepara os dados
brazil_5 = brazil[(brazil['geolocation_state']=='RO')|(brazil['geolocation_state']=='AM')|(brazil['geolocation_state']=='AC')|(brazil['geolocation_state']=='AP')|(brazil['geolocation_state']=='RR')]

In [20]:
# Figura
fig = px.scatter_mapbox(brazil_5, 
                        lat = "geolocation_lat", 
                        lon = "geolocation_lng", 
                        hover_name = "geolocation_city", 
                        hover_data = ["geolocation_state"],
                        color_discrete_sequence = ["fuchsia"], 
                        zoom = 2, 
                        center = dict(lat = -18, lon = -52), 
                        height = 300)

# Layout
fig.update_layout(mapbox_style = "open-street-map")
fig.update_layout(margin = {"r":0,"t":0,"l":0,"b":0})

# Gráfico
fig.show()

# Fim